In [14]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

## Data Import

In [15]:
data = pd.read_csv('./data_files/realtor-dataset-100k.csv')
data.head()

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        100000 non-null  object 
 1   price         100000 non-null  int64  
 2   bed           75050 non-null   float64
 3   bath          75112 non-null   float64
 4   acre_lot      85987 non-null   float64
 5   full_address  100000 non-null  object 
 6   street        99916 non-null   object 
 7   city          99948 non-null   object 
 8   state         100000 non-null  object 
 9   zip_code      99805 non-null   float64
 10  house_size    75082 non-null   float64
 11  sold_date     28745 non-null   object 
dtypes: float64(5), int64(1), object(6)
memory usage: 9.2+ MB


### Data Cleaning

In [17]:
data.isnull().sum()

status              0
price               0
bed             24950
bath            24888
acre_lot        14013
full_address        0
street             84
city               52
state               0
zip_code          195
house_size      24918
sold_date       71255
dtype: int64

Replacing Null Values in the sold_date column if there is no date most likely means the house hasn't been sold

In [18]:
data['sold_date'].replace(np.nan, 'Not Sold', inplace=True)
data.head()

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,Not Sold
1,for_sale,80000,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,Not Sold
2,for_sale,67000,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,Not Sold
3,for_sale,145000,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,Not Sold
4,for_sale,65000,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,Not Sold


I think the best way to replace the other Null vaules is to use the average values based on that cities location. However, with the missing values in the cities column we have to address those first  

In [19]:
data['city'].value_counts().count()

525

since City, Street, and Zip_code are all values unique to a specific house these cant be estimated. Best option for these is to replace with a filler such as "Missing". 

In [20]:
data['city'].fillna('Missing', inplace=True)
data['street'].fillna('Missing', inplace=True)
data['zip_code'].fillna('Missing', inplace=True)

How that all the unique identifiers have been replaced and now we are left with just house attributes these can be replaced with estimated numbers. To combat this for beds, baths, and acre_lot im going to use the mode number (Most Occurring) based on the city location. Most houses within a city are bulit the same or to a standard around the area. 

In [21]:
data.isnull().sum()

status              0
price               0
bed             24950
bath            24888
acre_lot        14013
full_address        0
street              0
city                0
state               0
zip_code            0
house_size      24918
sold_date           0
dtype: int64

In [22]:
city_bed_mode = data.groupby('city')['bed'].agg(pd.Series.mode).to_dict()

In [23]:
data['bed'].fillna(data['city'],inplace=True)

Replacing incosistant values with the overall average of 3 beds

In [24]:
for key, value in city_bed_mode.items():
    if not isinstance(value, float):
        city_bed_mode[key] = 3

Replacing the Placeholder city names with the correct most common bed value

In [25]:
data['bed'].replace(city_bed_mode, inplace=True)

Now Repeated Process for Bath, Acre, and House Size

In [26]:
city_bath_mode = data.groupby('city')['bath'].agg(pd.Series.mode).to_dict()
city_acre_mode = data.groupby('city')['acre_lot'].agg(pd.Series.mode).to_dict()
city_house_mode = data.groupby('city')['house_size'].agg(pd.Series.mode).to_dict()


In [27]:
data['bath'].fillna(data['city'],inplace=True)
data['acre_lot'].fillna(data['city'],inplace=True)
data['house_size'].fillna(data['city'],inplace=True)

In [28]:
## Bath Fix (most common overall is 2 baths)
for key, value in city_bath_mode.items():
    if not isinstance(value, float):
        city_bath_mode[key] = 2

## Acre Fix (most common overall is 0.09 baths)
for key, value in city_acre_mode.items():
    if not isinstance(value, float):
        city_acre_mode[key] = 0.09

## Acre Fix (most common overall is 0.09 baths)
for key, value in city_house_mode.items():
    if not isinstance(value, float):
        city_house_mode[key] = 1200

In [29]:
data['bath'].replace(city_bath_mode, inplace=True)
data['acre_lot'].replace(city_acre_mode, inplace=True)
data['house_size'].replace(city_house_mode, inplace=True)

In [30]:
data.isna().sum()

status          0
price           0
bed             0
bath            0
acre_lot        0
full_address    0
street          0
city            0
state           0
zip_code        0
house_size      0
sold_date       0
dtype: int64

In [31]:
data.duplicated().sum()

85141

Since there are duplicated houses for some reason with in the data I will only keep the 1st iteration of the house

In [32]:
data[data.duplicated(subset='full_address', keep=False)].sort_values('full_address')[:10]

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
49070,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
68374,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
36314,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
51831,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
50721,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
44000,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
35809,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
38260,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
41190,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold
29139,for_sale,75000,3.0,2.0,0.29,"0 Ashmun, Springfield, MA, 01101",0 Ashmun,Springfield,Massachusetts,1101,580.0,Not Sold


In [33]:
data.drop_duplicates(subset='full_address',keep='first', inplace=True)

Reseting index for inconsistancies after dropping duplicates

In [34]:
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

In [35]:
data['status'].value_counts()

for_sale          14715
ready_to_build        4
Name: status, dtype: int64

If a there is a sold date listed the status needs to be changed from "For_sale" to SOLD

In [36]:
type(data.iloc[14718]['sold_date'])

str

Shows all rows with a date attached to the sold_date column 

In [37]:
data[data.sold_date.str.contains(pat='[1-12]')]

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
403,for_sale,76900,3.0,2.0,0.08,"E 331 Apt E 331 Alborada Unit E331, Canovanas,...",E 331 Apt E 331 Alborada Unit E331,Canovanas,Puerto Rico,729,1200.0,2/28/2020
494,for_sale,110000,7.0,3.0,0.09,"B-2 Monte Elena, Dorado, PR, 00949",B-2 Monte Elena,Dorado,Puerto Rico,949,1192.0,6/28/2019
1009,for_sale,159000,3.0,2.0,0.09,"784 Carretera Unit B302, Caguas, PR, 00725",784 Carretera Unit B302,Caguas,Puerto Rico,725,1625.0,9/15/2021
1144,for_sale,380000,3.0,2.0,78.66,Km 12 5 Carretera 182 Km 12 5 Bo Guayabota Yab...,Km 12 5 Carretera 182 Km 12 5 Bo Guayabota Yab...,Yabucoa,Puerto Rico,767,1200.0,3/15/2021
2245,for_sale,950000,5.0,4.0,0.99,"46 Water Isle, Saint Thomas, VI, 00802",46 Water Isle,Saint Thomas,Virgin Islands,802,5000.0,10/11/2013
...,...,...,...,...,...,...,...,...,...,...,...,...
14711,for_sale,649900,4.0,3.0,0.37,"7 Reagan Rd, Milford, MA, 01757",7 Reagan Rd,Milford,Massachusetts,1757,2255.0,6/28/1994
14712,for_sale,379900,3.0,2.0,0.18,"159 Garden City Dr, Cranston, RI, 02920",159 Garden City Dr,Cranston,Rhode Island,2920,1497.0,7/7/2017
14714,for_sale,735000,3.0,2.0,0.90,"17 Sunset Rd, Bristol, RI, 02809",17 Sunset Rd,Bristol,Rhode Island,2809,1727.0,6/29/1999
14715,for_sale,785000,4.0,3.0,0.24,"7 Agawam Rd, Barrington, RI, 02806",7 Agawam Rd,Barrington,Rhode Island,2806,2166.0,9/23/2021


Updates all sold houses status to sold

In [38]:
for i, row in data.iterrows():
    if row['sold_date'] != 'Not Sold':
        data.loc[i, 'status'] = 'SOLD'


More even distribution between for_sale and sold still not great but a lot better

In [39]:
data['status'].value_counts()

for_sale          9420
SOLD              5295
ready_to_build       4
Name: status, dtype: int64

The full address is redundent since we already have city, state, zip_code, and street

In [40]:
data.drop(columns=['full_address'], inplace=True)

In [41]:
data

,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000,3.0,2.0,0.12,Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601,920.0,Not Sold
1,for_sale,80000,4.0,2.0,0.08,Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601,1527.0,Not Sold
2,for_sale,67000,2.0,1.0,0.15,556G 556-G 16 St,Juana Diaz,Puerto Rico,795,748.0,Not Sold
3,for_sale,145000,4.0,2.0,0.10,R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731,1800.0,Not Sold
4,for_sale,65000,6.0,2.0,0.05,14 Navarro,Mayaguez,Puerto Rico,680,1000.0,Not Sold
...,...,...,...,...,...,...,...,...,...,...,...
14714,SOLD,735000,3.0,2.0,0.90,17 Sunset Rd,Bristol,Rhode Island,2809,1727.0,6/29/1999
14715,SOLD,785000,4.0,3.0,0.24,7 Agawam Rd,Barrington,Rhode Island,2806,2166.0,9/23/2021
14716,for_sale,639900,2.0,2.0,0.09,15 Morgan Dr Unit 109,Natick,Massachusetts,1760,1705.0,Not Sold
14717,for_sale,465000,3.0,2.0,0.36,16 Park Ave,Framingham,Massachusetts,1701,1442.0,Not Sold


In [43]:
data.to_csv(path_or_buf='D:\Data_Science_Documents\Realtor_Data\data_files\clean_data.csv')